In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


In [63]:
df_train = pd.read_excel('LTF Challenge data with dictionary.xlsx', sheet_name='TrainData')
df_test = pd.read_excel('LTF Challenge data with dictionary.xlsx', sheet_name='TestData')


print("Original shapes:")
print("Train:", df_train.shape)
print("Test:", df_test.shape)

Original shapes:
Train: (47970, 105)
Test: (9986, 105)


In [64]:
# Calculate missing values percentage
missing_train = pd.DataFrame({
    'Column': df_train.columns,
    'Missing %': (df_train.isnull().sum() / len(df_train) * 100).round(2)
}).sort_values('Missing %', ascending=False)

missing_test = pd.DataFrame({
    'Column': df_test.columns,
    'Missing %': (df_test.isnull().sum() / len(df_test) * 100).round(2)
}).sort_values('Missing %', ascending=False)


Looking at the code, the issue is that you're trying to use `missing_train_filtered` and `missing_test_filtered` dataframes, but these haven't been created yet. You need to use `missing_train` and `missing_test` which were created in the previous cell.



In [68]:
print("Train Data Missing Values:")
for _, row in missing_train.iterrows():
    if row['Missing %'] > 0:
        print(f"{row['Column']}: {row['Missing %']}%")

print("\nTest Data Missing Values:")
for _, row in missing_test.iterrows():
    if row['Missing %'] > 0:
        print(f"{row['Column']}: {row['Missing %']}%")


Train Data Missing Values:
Avg_Disbursement_Amount_Bureau: 43.34%
Address type: 35.5%
Ownership: 35.5%
Location: 35.5%
mat_roof_Metal_GI_Asbestos_sheets: 0.35%
perc_Households_do_not_have_KCC_With_The_Credit_Limit_Of_50k: 0.35%
Households_with_improved_Sanitation_Facility: 0.35%
perc_of_Wall_material_with_Burnt_brick: 0.35%
Perc_of_house_with_6plus_room: 0.35%
perc_of_pop_living_in_hh_electricity: 0.35%
Women_15_19_Mothers_or_Pregnant_at_time_of_survey: 0.35%
perc_Households_with_Pucca_House_That_Has_More_Than_3_Rooms: 0.35%
Total_Land_For_Agriculture: 0.15%

Test Data Missing Values:
Target_Variable/Total Income: 100.0%
Avg_Disbursement_Amount_Bureau: 42.68%
Address type: 35.74%
Location: 35.74%
Ownership: 35.74%
perc_Households_do_not_have_KCC_With_The_Credit_Limit_Of_50k: 0.33%
Households_with_improved_Sanitation_Facility: 0.33%
perc_of_Wall_material_with_Burnt_brick: 0.33%
mat_roof_Metal_GI_Asbestos_sheets: 0.33%
perc_of_pop_living_in_hh_electricity: 0.33%
Women_15_19_Mothers_or_Pr

In [69]:
# Find numerical columns with missing data less than 1%
numeric_cols = df_train.select_dtypes(include=['int64', 'float64']).columns
low_missing_numeric = []

for col in numeric_cols:
    missing_pct = (df_train[col].isnull().sum() / len(df_train) * 100).round(2)
    if missing_pct < 1 and missing_pct > 0:
        low_missing_numeric.append((col, missing_pct))

print("Numerical columns with less than 1% missing data:")
for col, pct in low_missing_numeric:
    print(f"{col}: {pct}%")


Numerical columns with less than 1% missing data:
Total_Land_For_Agriculture: 0.15%
Perc_of_house_with_6plus_room: 0.35%
Women_15_19_Mothers_or_Pregnant_at_time_of_survey: 0.35%
perc_of_pop_living_in_hh_electricity: 0.35%
perc_Households_with_Pucca_House_That_Has_More_Than_3_Rooms: 0.35%
mat_roof_Metal_GI_Asbestos_sheets: 0.35%
perc_of_Wall_material_with_Burnt_brick: 0.35%
Households_with_improved_Sanitation_Facility: 0.35%
perc_Households_do_not_have_KCC_With_The_Credit_Limit_Of_50k: 0.35%


In [72]:
# Create IterativeImputer with RandomForestRegressor
rf_imputer = IterativeImputer(
    estimator=RandomForestRegressor(n_estimators=100, random_state=42),
    random_state=42,
    max_iter=10
)

# Get columns from low_missing_numeric list
columns_to_impute = [col for col, _ in low_missing_numeric]

# Store original data and missing masks
original_data = df_train[columns_to_impute].copy()
missing_masks = df_train[columns_to_impute].isnull()

# Impute missing values
df_train[columns_to_impute] = rf_imputer.fit_transform(df_train[columns_to_impute])
df_test[columns_to_impute] = rf_imputer.transform(df_test[columns_to_impute])

# Calculate statistics for only the imputed values
comparison = {}
for col in columns_to_impute:
    # Get values that were originally missing
    imputed_values = df_train[col][missing_masks[col]]

    comparison[col] = {
        'Original Median (non-missing)': original_data[col].median(),
        'Mean of Imputed Values': imputed_values.mean()
    }

# Create and display comparison DataFrame
comparison_df = pd.DataFrame(comparison).T
print("\nComparison of original medians vs means of imputed values:")
print(comparison_df)

# Verify imputation worked




Comparison of original medians vs means of imputed values:
                                                    Original Median (non-missing)  \
Total_Land_For_Agriculture                                                 10.000   
Perc_of_house_with_6plus_room                                               1.170   
Women_15_19_Mothers_or_Pregnant_at_time_of_survey                           5.800   
perc_of_pop_living_in_hh_electricity                                       98.945   
perc_Households_with_Pucca_House_That_Has_More_...                          4.300   
mat_roof_Metal_GI_Asbestos_sheets                                           8.250   
perc_of_Wall_material_with_Burnt_brick                                     39.910   
Households_with_improved_Sanitation_Facility                               71.300   
perc_Households_do_not_have_KCC_With_The_Credit...                          1.750   

                                                    Mean of Imputed Values  
Total_Land_F

In [98]:
print("\nMissing values after imputation:")
print("\nTrain data:")
print(df_train[columns_to_impute].isnull().sum())
print("\nTest data:")
print(df_test[columns_to_impute].isnull().sum())


Missing values after imputation:

Train data:
Total_Land_For_Agriculture                                      0
Perc_of_house_with_6plus_room                                   0
Women_15_19_Mothers_or_Pregnant_at_time_of_survey               0
perc_of_pop_living_in_hh_electricity                            0
perc_Households_with_Pucca_House_That_Has_More_Than_3_Rooms     0
mat_roof_Metal_GI_Asbestos_sheets                               0
perc_of_Wall_material_with_Burnt_brick                          0
Households_with_improved_Sanitation_Facility                    0
perc_Households_do_not_have_KCC_With_The_Credit_Limit_Of_50k    0
dtype: int64

Test data:
Total_Land_For_Agriculture                                      0
Perc_of_house_with_6plus_room                                   0
Women_15_19_Mothers_or_Pregnant_at_time_of_survey               0
perc_of_pop_living_in_hh_electricity                            0
perc_Households_with_Pucca_House_That_Has_More_Than_3_Rooms     0
mat_

In [83]:
df_train_locked = df_train.copy(deep=True)
df_test_locked = df_test.copy(deep=True)

In [73]:
missing_train = pd.DataFrame({
    'Column': df_train.columns,
    'Missing %': (df_train.isnull().sum() / len(df_train) * 100).round(2)
}).sort_values('Missing %', ascending=False)

missing_test = pd.DataFrame({
    'Column': df_test.columns,
    'Missing %': (df_test.isnull().sum() / len(df_test) * 100).round(2)
}).sort_values('Missing %', ascending=False)

print("Train Data Missing Values:")
for _, row in missing_train.iterrows():
    if row['Missing %'] > 0:
        print(f"{row['Column']}: {row['Missing %']}%")

print("\nTest Data Missing Values:")
for _, row in missing_test.iterrows():
    if row['Missing %'] > 0:
        print(f"{row['Column']}: {row['Missing %']}%")

Train Data Missing Values:
Avg_Disbursement_Amount_Bureau: 43.34%
Address type: 35.5%
Ownership: 35.5%
Location: 35.5%

Test Data Missing Values:
Target_Variable/Total Income: 100.0%
Avg_Disbursement_Amount_Bureau: 42.68%
Address type: 35.74%
Location: 35.74%
Ownership: 35.74%


In [157]:
non_numeric_columns = df_train.select_dtypes(exclude=['int64', 'float64']).columns
print("\nNon-numeric columns in the dataset:")
for col in non_numeric_columns:
    print(f"- {col}")


Non-numeric columns in the dataset:
- State
- REGION
- CITY
- DISTRICT
- VILLAGE
- MARITAL_STATUS
- Location
- Address type
- Ownership
- K022-Nearest Mandi Name
- K022-Ambient temperature (min & max)
- R022-Ambient temperature (min & max)
- K021-Ambient temperature (min & max)
- R021-Ambient temperature (min & max)
- R020-Ambient temperature (min & max)
- Kharif Seasons  Type of soil in 2022
- Kharif Seasons  Type of water bodies in hectares 2022
- Kharif Seasons  Agro Ecological Sub Zone in 2022
- Rabi Seasons Type of soil in 2022
- Rabi Seasons Type of water bodies in hectares 2022
- Rabi Seasons Agro Ecological Sub Zone in 2022
- Rabi Seasons Type of soil in 2021
- Rabi Seasons Type of water bodies in hectares 2021
- Rabi Seasons Agro Ecological Sub Zone in 2021
- Kharif Seasons Type of soil in 2021
- Kharif Seasons Type of water bodies in hectares 2021
- Kharif Seasons Agro Ecological Sub Zone in 2021
- Kharif Seasons Type of soil in 2020
- Kharif Seasons Type of water bodies in 

In [158]:
for col in non_numeric_columns:
    unique_values = df_train[col].nunique()
    print(f"{col}: {unique_values}")


State: 17
REGION: 5
CITY: 2721
DISTRICT: 405
VILLAGE: 5650
MARITAL_STATUS: 3
Location: 30634
Address type: 3
Ownership: 3
K022-Nearest Mandi Name: 890
K022-Ambient temperature (min & max): 1304
R022-Ambient temperature (min & max): 1311
K021-Ambient temperature (min & max): 1307
R021-Ambient temperature (min & max): 1307
R020-Ambient temperature (min & max): 1307
Kharif Seasons  Type of soil in 2022: 8
Kharif Seasons  Type of water bodies in hectares 2022: 22
Kharif Seasons  Agro Ecological Sub Zone in 2022: 12
Rabi Seasons Type of soil in 2022: 8
Rabi Seasons Type of water bodies in hectares 2022: 22
Rabi Seasons Agro Ecological Sub Zone in 2022: 12
Rabi Seasons Type of soil in 2021: 8
Rabi Seasons Type of water bodies in hectares 2021: 22
Rabi Seasons Agro Ecological Sub Zone in 2021: 12
Kharif Seasons Type of soil in 2021: 8
Kharif Seasons Type of water bodies in hectares 2021: 22
Kharif Seasons Agro Ecological Sub Zone in 2021: 12
Kharif Seasons Type of soil in 2020: 8
Kharif Seaso

In [159]:
value_counts = {}
for col in df_train.columns:
    value_counts[col] = df_train[col].nunique()

low_unique = pd.Series(value_counts)
low_unique = low_unique[low_unique < 20].sort_values()

print("Columns with less than 20 unique values:")
for col, count in low_unique.items():
    print(f"{col}: {count} unique values")

Columns with less than 20 unique values:
SEX: 2 unique values
K022-Village category based on Agri parameters (Good, Average, Poor): 2 unique values
R022-Village category based on Agri parameters (Good, Average, Poor): 2 unique values
Address type: 3 unique values
Ownership: 3 unique values
K022-Village category based on socio-economic parameters (Good, Average, Poor): 3 unique values
MARITAL_STATUS: 3 unique values
 Village category based on socio-economic parameters (Good, Average, Poor): 3 unique values
REGION: 5 unique values
Kharif Seasons Type of soil in 2021: 8 unique values
Rabi Seasons Type of soil in 2022: 8 unique values
Kharif Seasons  Type of soil in 2022: 8 unique values
Kharif Seasons Type of soil in 2020: 8 unique values
Rabi Seasons Type of soil in 2020: 8 unique values
Rabi Seasons Type of soil in 2021: 8 unique values
Rabi Seasons Agro Ecological Sub Zone in 2022: 12 unique values
Kharif Seasons Agro Ecological Sub Zone in 2020: 12 unique values
Rabi Seasons Agro Ecol

In [150]:
for col in low_unique.index:
    print(f"\nUnique values and their counts in {col}:")
    value_counts = df_train[col].value_counts()
    for val, count in value_counts.items():
        print(f"{val}: {count} records")
        
print(len(low_unique))



Unique values and their counts in SEX:
M: 43295 records
F: 4675 records

Unique values and their counts in K022-Village category based on Agri parameters (Good, Average, Poor):
0.0: 36513 records
1.0: 11457 records

Unique values and their counts in R022-Village category based on Agri parameters (Good, Average, Poor):
0.0: 41480 records
1.0: 6490 records

Unique values and their counts in Address type:
Permanent Address: 26213 records
Current Address: 3964 records
Both Addresses: 763 records

Unique values and their counts in Ownership:
Owned: 29462 records
Parental: 1469 records
Rented: 9 records

Unique values and their counts in K022-Village category based on socio-economic parameters (Good, Average, Poor):
0.0: 29221 records
1.0: 17824 records
2.0: 925 records

Unique values and their counts in MARITAL_STATUS:
M: 44168 records
S: 3798 records
NK: 4 records

Unique values and their counts in  Village category based on socio-economic parameters (Good, Average, Poor):
0.0: 29221 reco

In [137]:
ordered_string_columns = ["K022-Village category based on Agri parameters (Good, Average, Poor)",
                          "R022-Village category based on Agri parameters (Good, Average, Poor)",
                          "K022-Village category based on socio-economic parameters (Good, Average, Poor)",
                          " Village category based on socio-economic parameters (Good, Average, Poor)"]

for col in ordered_string_columns:
    #df_train[col] = df_train_old[col]
    #df_test[col] = df_test_old[col]
    print(df_train[col].dtype)
    df_train[col] = df_train[col].astype(str).str.strip()
    df_test[col] = df_test[col].astype(str).str.strip()
    print(df_train[col].dtype)

object
object
object
object
object
object
object
object


In [138]:
"""from sklearn.preprocessing import OrdinalEncoder

# Create ordinal encoder
ord_encoder = OrdinalEncoder(
    categories=[["Poor", "Average", "Good"]] * len(ordered_string_columns),
)

for col in ordered_string_columns:
    train_unique = set(df_train[col].dropna().unique())
    test_unique = set(df_test[col].dropna().unique())
    if not test_unique.issubset(train_unique):
        print(f"⚠️ Unexpected category in test column: {col}")

# Fit and transform the training data
df_train[ordered_string_columns] = ord_encoder.fit_transform(df_train[ordered_string_columns])

# Transform the test data using the same encoder
df_test[ordered_string_columns] = ord_encoder.transform(df_test[ordered_string_columns])"""

Applied ordinal encoding to convert ordered categorical values in the specified columns to numerical values. The encoding preserves the ordinal relationship between categories (e.g. Good > Average > Poor).

In [139]:
# Verify the encoding
for col in ordered_string_columns:
    print(f"\nUnique values in {col} after encoding:")
    print(df_train[col].unique())



Unique values in K022-Village category based on Agri parameters (Good, Average, Poor) after encoding:
[0. 1.]

Unique values in R022-Village category based on Agri parameters (Good, Average, Poor) after encoding:
[0. 1.]

Unique values in K022-Village category based on socio-economic parameters (Good, Average, Poor) after encoding:
[0. 1. 2.]

Unique values in  Village category based on socio-economic parameters (Good, Average, Poor) after encoding:
[0. 1. 2.]


In [141]:
value_counts = {}
for col in non_numeric_columns:
    value_counts[col] = df_train[col].nunique()

low_unique = pd.Series(value_counts)
low_unique = low_unique[low_unique >= 20].sort_values()

print("Columns with less than 20 unique values:")
for col, count in low_unique.items():
    print(f"{col}: {count} unique values")

Columns with less than 20 unique values:
Rabi Seasons Type of water bodies in hectares 2021: 22 unique values
Kharif Seasons Type of water bodies in hectares 2021: 22 unique values
Rabi Seasons Type of water bodies in hectares 2020: 22 unique values
Kharif Seasons Type of water bodies in hectares 2020: 22 unique values
Kharif Seasons  Type of water bodies in hectares 2022: 22 unique values
Rabi Seasons Type of water bodies in hectares 2022: 22 unique values
DISTRICT: 405 unique values
K022-Nearest Mandi Name: 890 unique values
K022-Ambient temperature (min & max): 1304 unique values
K021-Ambient temperature (min & max): 1307 unique values
R021-Ambient temperature (min & max): 1307 unique values
R020-Ambient temperature (min & max): 1307 unique values
R022-Ambient temperature (min & max): 1311 unique values
CITY: 2721 unique values
VILLAGE: 5650 unique values
Location: 30634 unique values


In [161]:
left_low = low_unique[~low_unique.index.isin(ordered_string_columns)]
print(len(left_low))
print(len(low_unique))
print("\nColumns with less than 20 unique values:")
for col in left_low.index:
    print(f"\n{col}")


18
22

Columns with less than 20 unique values:

SEX

Address type

Ownership

MARITAL_STATUS

REGION

Kharif Seasons Type of soil in 2021

Rabi Seasons Type of soil in 2022

Kharif Seasons  Type of soil in 2022

Kharif Seasons Type of soil in 2020

Rabi Seasons Type of soil in 2020

Rabi Seasons Type of soil in 2021

Rabi Seasons Agro Ecological Sub Zone in 2022

Kharif Seasons Agro Ecological Sub Zone in 2020

Rabi Seasons Agro Ecological Sub Zone in 2021

Kharif Seasons  Agro Ecological Sub Zone in 2022

Kharif Seasons Agro Ecological Sub Zone in 2021

Rabi Seasons Agro Ecological Sub Zone in 2020

State


In [ ]:
"""df_test["SEX"] = df_test_old["SEX"]
df_train["SEX"] = df_train_old["SEX"]
df_train['SEX'] = df_train['SEX'].map({'M': 1, 'F': 0})
df_test['SEX'] = df_test['SEX'].map({'M': 1, 'F': 0})"""

In [ ]:
print("Unique values in SEX column after encoding:")
print(df_train['SEX'].unique())


In [ ]:
"""
Address type: 3 unique values


MARITAL_STATUS: 3 unique values
"""

In [163]:
list_ = ["Address type",
        "Ownership",
        "MARITAL_STATUS",
        "REGION",
        "State",
        "Kharif Seasons Type of soil in 2021",
        "Rabi Seasons Type of soil in 2022",
        "Kharif Seasons  Type of soil in 2022",
        "Kharif Seasons Type of soil in 2020",
        "Rabi Seasons Type of soil in 2020",
        "Rabi Seasons Type of soil in 2021",
        "Rabi Seasons Agro Ecological Sub Zone in 2022",
        "Kharif Seasons Agro Ecological Sub Zone in 2020",
        "Rabi Seasons Agro Ecological Sub Zone in 2021",
        "Kharif Seasons  Agro Ecological Sub Zone in 2022",
        "Kharif Seasons Agro Ecological Sub Zone in 2021",
        "Rabi Seasons Agro Ecological Sub Zone in 2020",
        ]

In [164]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
le = LabelEncoder()

# Encode each categorical column
for col in list_:
    df_train[col] = le.fit_transform(df_train[col].astype(str))
    df_test[col] = le.transform(df_test[col].astype(str))

In [168]:
# Verify encoding
df_train[list_].head()

,Address type,Ownership,MARITAL_STATUS,REGION,State,Kharif Seasons Type of soil in 2021,Rabi Seasons Type of soil in 2022,Kharif Seasons Type of soil in 2022,Kharif Seasons Type of soil in 2020,Rabi Seasons Type of soil in 2020,Rabi Seasons Type of soil in 2021,Rabi Seasons Agro Ecological Sub Zone in 2022,Kharif Seasons Agro Ecological Sub Zone in 2020,Rabi Seasons Agro Ecological Sub Zone in 2021,Kharif Seasons Agro Ecological Sub Zone in 2022,Kharif Seasons Agro Ecological Sub Zone in 2021,Rabi Seasons Agro Ecological Sub Zone in 2020
0,3,3,0,0,9,2,2,2,2,2,2,1,1,1,1,1,1
1,3,3,0,1,2,4,4,4,4,4,4,2,2,2,2,2,2
2,3,3,0,0,9,2,2,2,2,2,2,0,0,0,0,0,0
3,3,3,0,4,10,7,7,7,7,7,7,3,3,3,3,3,3
4,3,3,0,0,9,2,2,2,2,2,2,1,1,1,1,1,1


In [169]:
non_numeric_columns = df_train.select_dtypes(exclude=['int64', 'float64']).columns
print("\nNon-numeric columns in the dataset:")
for col in non_numeric_columns:
    print(f"- {col}")


Non-numeric columns in the dataset:
- CITY
- DISTRICT
- VILLAGE
- Location
- K022-Nearest Mandi Name
- K022-Ambient temperature (min & max)
- R022-Ambient temperature (min & max)
- K021-Ambient temperature (min & max)
- R021-Ambient temperature (min & max)
- R020-Ambient temperature (min & max)
- Kharif Seasons  Type of water bodies in hectares 2022
- Rabi Seasons Type of water bodies in hectares 2022
- Rabi Seasons Type of water bodies in hectares 2021
- Kharif Seasons Type of water bodies in hectares 2021
- Kharif Seasons Type of water bodies in hectares 2020
- Rabi Seasons Type of water bodies in hectares 2020


In [171]:
min_max_list = [
                "K022-Ambient temperature (min & max)",
                "R022-Ambient temperature (min & max)",
                "K021-Ambient temperature (min & max)",
                "R021-Ambient temperature (min & max)",
                "R020-Ambient temperature (min & max)",
                ]

for col in min_max_list:
    df_test[[col + " min", col + " max"]] = df_test[col].str.replace(' ', '', regex=False).str.split('/', expand=True)
    df_train[[col + " min", col + " max"]] = df_train[col].str.replace(' ', '', regex=False).str.split('/', expand=True)

    df_test[col + " min"] = df_test[col + " min"].astype(float)
    df_test[col + " max"] = df_test[col + " max"].astype(float)

    df_train[col + " min"] = df_train[col + " min"].astype(float)
    df_train[col + " max"] = df_train[col + " max"].astype(float)



In [176]:
for col in min_max_list:
    df_train.drop(col, axis=1, inplace=True)
    df_test.drop(col, axis=1, inplace=True)

In [179]:
non_numeric_columns = df_train.select_dtypes(exclude=['int64', 'float64']).columns
print("\nNon-numeric columns in the dataset:")
for col in non_numeric_columns:
    print(f"- {col}")

value_counts = {}
for col in non_numeric_columns:
    value_counts[col] = df_train[col].nunique()

low_unique = pd.Series(value_counts)
low_unique = low_unique.sort_values()

print("Columns with less than 20 unique values:")
for col, count in low_unique.items():
    print(f"{col}: {count} unique values")


Non-numeric columns in the dataset:
- CITY
- DISTRICT
- VILLAGE
- Location
- K022-Nearest Mandi Name
- Kharif Seasons  Type of water bodies in hectares 2022
- Rabi Seasons Type of water bodies in hectares 2022
- Rabi Seasons Type of water bodies in hectares 2021
- Kharif Seasons Type of water bodies in hectares 2021
- Kharif Seasons Type of water bodies in hectares 2020
- Rabi Seasons Type of water bodies in hectares 2020
Columns with less than 20 unique values:
Rabi Seasons Type of water bodies in hectares 2022: 22 unique values
Rabi Seasons Type of water bodies in hectares 2021: 22 unique values
Kharif Seasons  Type of water bodies in hectares 2022: 22 unique values
Kharif Seasons Type of water bodies in hectares 2021: 22 unique values
Kharif Seasons Type of water bodies in hectares 2020: 22 unique values
Rabi Seasons Type of water bodies in hectares 2020: 22 unique values
DISTRICT: 405 unique values
K022-Nearest Mandi Name: 890 unique values
CITY: 2721 unique values
VILLAGE: 5650 u

In [180]:
water_bodies_columns = [
    'Rabi Seasons Type of water bodies in hectares 2022',
    'Rabi Seasons Type of water bodies in hectares 2021',
    'Kharif Seasons  Type of water bodies in hectares 2022',
    'Kharif Seasons Type of water bodies in hectares 2021',
    'Kharif Seasons Type of water bodies in hectares 2020',
    'Rabi Seasons Type of water bodies in hectares 2020'
]

# Initialize LabelEncoder
le = LabelEncoder()

# Encode each column
for col in water_bodies_columns:
    df_train[col] = le.fit_transform(df_train[col].astype(str))
    df_test[col] = le.transform(df_test[col].astype(str))

In [181]:
# Verify encoding
for col in water_bodies_columns:
    print(f"\nUnique values in {col} after encoding:")
    print(df_train[col].unique())



Unique values in Rabi Seasons Type of water bodies in hectares 2022 after encoding:
[18  8 21 15  5 12  0  1 10 13 16  9  3 14 19 20  4 11 17  6  7  2]

Unique values in Rabi Seasons Type of water bodies in hectares 2021 after encoding:
[18  8 21 15  5 12  0  1 10 13 16  9  3 14 19 20  4 11 17  6  7  2]

Unique values in Kharif Seasons  Type of water bodies in hectares 2022 after encoding:
[18  8 21 15  5 12  0  1 10 13 16  9  3 14 19 20  4 11 17  6  7  2]

Unique values in Kharif Seasons Type of water bodies in hectares 2021 after encoding:
[18  8 21 15  5 12  0  1 10 13 16  9  3 14 19 20  4 11 17  6  7  2]

Unique values in Kharif Seasons Type of water bodies in hectares 2020 after encoding:
[18  8 21 15  5 12  0  1 10 16 13  3  9 14 19 20  4 11  6  7 17  2]

Unique values in Rabi Seasons Type of water bodies in hectares 2020 after encoding:
[18  8 21 15  5 12  0  1 10 13 16  9  3 14 19 20  4 11 17  6  7  2]


In [182]:
non_numeric_columns = df_train.select_dtypes(exclude=['int64', 'float64']).columns
print("\nNon-numeric columns in the dataset:")
for col in non_numeric_columns:
    print(f"- {col}")

value_counts = {}
for col in non_numeric_columns:
    value_counts[col] = df_train[col].nunique()

low_unique = pd.Series(value_counts)
low_unique = low_unique.sort_values()

print("Columns with less than 20 unique values:")
for col, count in low_unique.items():
    print(f"{col}: {count} unique values")


Non-numeric columns in the dataset:
- CITY
- DISTRICT
- VILLAGE
- Location
- K022-Nearest Mandi Name
Columns with less than 20 unique values:
DISTRICT: 405 unique values
K022-Nearest Mandi Name: 890 unique values
CITY: 2721 unique values
VILLAGE: 5650 unique values
Location: 30634 unique values


In [183]:
for col in ['DISTRICT', 'K022-Nearest Mandi Name', 'CITY']:
    mean_map = df_train.groupby(col)['Target_Variable/Total Income'].mean()
    df_train[col] = df_train[col].map(mean_map)
    df_test[col] = df_test[col].map(mean_map)  # will produce NaN for unseen categories

In [184]:
non_numeric_columns = df_train.select_dtypes(exclude=['int64', 'float64']).columns
print("\nNon-numeric columns in the dataset:")
for col in non_numeric_columns:
    print(f"- {col}")

value_counts = {}
for col in non_numeric_columns:
    value_counts[col] = df_train[col].nunique()

low_unique = pd.Series(value_counts)
low_unique = low_unique.sort_values()

print("Columns with less than 20 unique values:")
for col, count in low_unique.items():
    print(f"{col}: {count} unique values")


Non-numeric columns in the dataset:
- VILLAGE
- Location
Columns with less than 20 unique values:
VILLAGE: 5650 unique values
Location: 30634 unique values


In [187]:
"""Avg_Disbursement_Amount_Bureau: 43.34%
Address type: 35.5%
Ownership: 35.5%
Location: 35.5%"""

df_train["Ownership"] = df_train_old["Ownership"]
df_test["Ownership"] = df_test_old["Ownership"]
df_train["Address type"] = df_train_old["Address type"]
df_test["Address type"] = df_test_old["Address type"]

non_numeric_columns = df_train.select_dtypes(exclude=['int64', 'float64']).columns
print("\nNon-numeric columns in the dataset:")
for col in non_numeric_columns:
    print(f"- {col}")

value_counts = {}
for col in non_numeric_columns:
    value_counts[col] = df_train[col].nunique()

low_unique = pd.Series(value_counts)
low_unique = low_unique.sort_values()

print("Columns with less than 20 unique values:")
for col, count in low_unique.items():
    print(f"{col}: {count} unique values")


Non-numeric columns in the dataset:
- VILLAGE
- Location
- Address type
- Ownership
Columns with less than 20 unique values:
Ownership: 3 unique values
Address type: 3 unique values
VILLAGE: 5650 unique values
Location: 30634 unique values


In [188]:
for col in ['Address type', 'Ownership', 'Location']:
    df_train[col] = df_train[col].fillna('Missing')
    df_test[col] = df_test[col].fillna('Missing')

In [189]:
# Create label encoder for Address type and Ownership
le_address = LabelEncoder()
le_ownership = LabelEncoder()

# Fit and transform Address type
df_train['Address type'] = le_address.fit_transform(df_train['Address type'])
df_test['Address type'] = le_address.transform(df_test['Address type'])

# Fit and transform Ownership 
df_train['Ownership'] = le_ownership.fit_transform(df_train['Ownership'])
df_test['Ownership'] = le_ownership.transform(df_test['Ownership'])

In [190]:
# Verify encoding
print("\nUnique values in Address type after encoding:")
print(df_train['Address type'].unique())
print("\nUnique values in Ownership after encoding:")
print(df_train['Ownership'].unique())



Unique values in Address type after encoding:
[2 3 1 0]

Unique values in Ownership after encoding:
[0 1 2 3]


In [194]:
for col in df_train.columns:
    if df_train[col].isnull().sum() > 0 or df_test[col].isnull().sum() > 0:
        print(col)
        print("\nTrain data:")
        print(df_train[col].isnull().sum())
        print("\nTest data:")
        print(df_test[col].isnull().sum())

SEX

Train data:
0

Test data:
2
CITY

Train data:
0

Test data:
99
DISTRICT

Train data:
0

Test data:
1
Avg_Disbursement_Amount_Bureau

Train data:
20790

Test data:
4262
K022-Nearest Mandi Name

Train data:
0

Test data:
8
Target_Variable/Total Income

Train data:
0

Test data:
9986
